# Generate Images 3D
Jan 22, 2021

In [1]:
import argparse
import os
import sys

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data

from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

from torch.utils.data import DataLoader, TensorDataset

import time
from datetime import datetime
import glob
import pickle
import yaml
import collections

In [2]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/4_basic_3d_GAN/1_main_code/DDP/')
from utils import *

## Build Model structure

In [36]:
def f_load_config(config_file):
    with open(config_file) as f:
        config = yaml.load(f, Loader=yaml.SafeLoader)
    return config

# def f_init_gdict(gdict,config_dict):
#     ''' Initialize the global dictionary gdict with values in config file'''
#     keys1=['workers','nc','nz','ngf','ndf','beta1','kernel_size','stride','g_padding','d_padding','flip_prob']
#     keys2=['image_size','checkpoint_size','num_imgs','ip_fname','op_loc']
#     for key in keys1: gdict[key]=config_dict['training'][key]
#     for key in keys2: gdict[key]=config_dict['data'][key]
        
        
def f_manual_add_argparse():
    ''' use only in jpt notebook'''
    args=argparse.Namespace()
    args.config='config_3dgan.yaml'
    args.mode='fresh'
    args.ip_fldr=''
#     args.local_rank=0
    args.facility='cori'
    args.distributed=False
#     args.mode='continue'
#     args.ip_fldr='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20201211_093818_nb_test/'
    
    return args
        
# def f_init_gdict(gdict,config_file):
#     ''' Create global dictionary gdict from args and config file'''
    
#     ## read config file
#     with open(config_file) as f:
#         config_dict= yaml.load(f, Loader=yaml.SafeLoader)
        
#     gdict=config_dict['parameters']
    
#     args_dict=vars(args)
#     ## Add args variables to gdict
#     for key in args_dict.keys():
#         gdict[key]=args_dict[key]

        
        
        
def f_init_gdict(args,gdict):
    ''' Create global dictionary gdict from args and config file'''
    
    ## read config file
    config_file=args.config
    with open(config_file) as f:
        config_dict= yaml.load(f, Loader=yaml.SafeLoader)
        
    gdict=config_dict['parameters']

    args_dict=vars(args)
    ## Add args variables to gdict
    for key in args_dict.keys():
        gdict[key]=args_dict[key]
    return gdict

In [30]:
def f_gen_images(gdict,netG,optimizerG,ip_fname,op_loc,op_strg='inf_img_',op_size=500):
    '''Generate images for best saved models
     Arguments: gdict, netG, optimizerG, 
                 ip_fname: name of input file
                op_strg: [string name for output file]
                op_size: Number of images to generate
    '''

    nz,device=gdict['nz'],gdict['device']

    try:# handling cpu vs gpu
        if torch.cuda.is_available(): checkpoint=torch.load(ip_fname)
        else: checkpoint=torch.load(ip_fname,map_location=torch.device('cpu'))
    except Exception as e:
        print(e)
        print("skipping generation of images for ",ip_fname)
        return
    
    ## Load checkpoint
    if gdict['multi-gpu']:
        netG.module.load_state_dict(checkpoint['G_state'])
    else:
        netG.load_state_dict(checkpoint['G_state'])
    
    ## Load other stuff
    iters=checkpoint['iters']
    epoch=checkpoint['epoch']
    optimizerG.load_state_dict(checkpoint['optimizerG_state_dict'])
    
    # Generate batch of latent vectors
    noise = torch.randn(op_size, 1, 1, 1, nz, device=device)
    # Generate fake image batch with G
    netG.eval() ## This is required before running inference
    with torch.no_grad(): ## This is important. fails without it for multi-gpu
        gen = netG(noise)
        gen_images=gen.detach().cpu().numpy()
        print(gen_images.shape)
    
    op_fname='%s_epoch-%s_step-%s.npy'%(op_strg,epoch,iters)
    np.save(op_loc+op_fname,gen_images)

    print("Image saved in ",op_fname)

In [31]:
gdict

{'ip_fname': '/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/raw_data/3d_data/64cube/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy',
 'op_loc': '/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/',
 'image_size': 64,
 'num_imgs': 90000,
 'workers': 2,
 'nc': 1,
 'nz': 64,
 'ngf': 64,
 'ndf': 64,
 'beta1': 0.5,
 'kernel_size': 5,
 'stride': 2,
 'g_padding': 2,
 'd_padding': 2,
 'flip_prob': 0.01,
 'bns': 50,
 'checkpoint_size': 1,
 'batch_size': 8,
 'epochs': 1,
 'learn_rate': 0.0002,
 'op_size': 32,
 'deterministic': False,
 'seed': 234373,
 'lambda_spec_mean': 0.1,
 'lambda_spec_var': 0.1,
 'lambda_fm': 0.0,
 'lambda_gp': 0.0,
 'grad_clip': 1.0,
 'save_steps_list': [5, 10],
 'run_suffix': 'bs8_lr0.0001_nodes8',
 'description': '3d GAN: DDP with new loss',
 'config': '1_main_code/DDP/config_3dgan_summit.yaml',
 'ngpu': 0,
 'batchsize': 128,
 'device': device(type='cpu'),
 'multi-gpu': False}

In [38]:
if __name__=="__main__":
    torch.backends.cudnn.benchmark=True
    t0=time.time()
    #################################
    f_manual_add_argparse()
    ### Set up ###

    # Initilize variables    
    gdict={}
    gdict=f_init_gdict(args,gdict)
    print(gdict)
    ## Add args variables to gdict
    for key in ['ngpu']:
        gdict[key]=vars(args)[key]
    gdict['device']=torch.device("cuda" if (torch.cuda.is_available() and gdict['ngpu'] > 0) else "cpu")
    gdict['ngpu']=torch.cuda.device_count()
    gdict['multi-gpu']=True if (gdict['device'].type == 'cuda') and (gdict['ngpu'] > 1) else False 

    print("Building GAN networks")
    # Create Generator
    netG = Generator(gdict).to(gdict['device'])
    netG.apply(weights_init)
    #     print(netG)
    # summary(netG,(1,1,64))
    
    print("Number of GPUs used %s"%(gdict['ngpu']))
    if (gdict['multi-gpu']):
        netG = nn.DataParallel(netG, list(range(gdict['ngpu'])))

    optimizerG = optim.Adam(netG.parameters(), lr=gdict['learn_rate'], betas=(gdict['beta1'], 0.999),eps=1e-7)


{'ip_fname': '/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/raw_data/3d_data/64cube/dataset1_smoothing_const_params_64cube_100k/norm_1_train_val.npy', 'op_loc': '/gpfs/alpine/ast153/proj-shared/venkitesh/Cosmogan/data/results_pytorch/3d/', 'image_size': 64, 'num_imgs': 90000, 'workers': 2, 'nc': 1, 'nz': 64, 'ngf': 64, 'ndf': 64, 'beta1': 0.5, 'kernel_size': 5, 'stride': 2, 'g_padding': 2, 'd_padding': 2, 'flip_prob': 0.01, 'bns': 50, 'checkpoint_size': 1, 'batch_size': 8, 'epochs': 1, 'learn_rate': 0.0001, 'op_size': 32, 'deterministic': False, 'seed': 234373, 'lambda_spec_mean': 0.1, 'lambda_spec_var': 0.1, 'lambda_fm': 0.0, 'lambda_gp': 0.0, 'grad_clip': 1.0, 'save_steps_list': [5, 10], 'run_suffix': 'bs8_lr0.0001_nodes8', 'description': '3d GAN: DDP with new loss', 'config': '1_main_code/DDP/config_3dgan_summit.yaml', 'ngpu': 1, 'batchsize': 128}
Building GAN networks
Number of GPUs used 0


## Run Inference

In [34]:
ls /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/

20210122_061039_3d_l0.1_80k/
20210123_050147_3d_l0.5_80k/
20210123_050147_3d_l2.0_80k/
20210128_200536_3d_l0.5_80k/
20210421_135443_bs64_lr0.0002_nodes8/
20210422_123705_bs4_lr0.00004_nodes8/
20210422_125528_bs8_lr0.00004_nodes8/
20210422_73038_bs64_lr0.00008_nodes8/
20210422_92518_bs32_lr0.00004_nodes8/
20210422_92953_bs64_lr0.00004_nodes8/
20210426_83932_bs8_lr0.0002_nodes8/
20210426_85259_bs8_lr0.0008_nodes8/
20210426_85502_bs8_lr0.0016_nodes8/
20210427_103955_bs8_lr0.0001_nodes8/
20210427_105753_bs8_lr0.001_nodes8_spec10.0/
20210427_134632_bs8_lr0.0003_nodes8/
20210427_140553_bs16_lr0.0003_nodes8/
20210428_62843_bs8_lr0.0003_nodes8/


In [8]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d_cGAN/'
# fldr='20210227_050213_3dcgan_predict_0.65_m2'
# op_loc=main_dir+fldr+'/images/'
# ip_fname=main_dir+fldr+'/models/checkpoint_22010.tar'
# f_gen_images(gdict,netG,optimizerG,ip_fname,op_loc,op_strg='inference_spec',op_size=500)

[Errno 2] No such file or directory: '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210121_133144_3d_l0.5/models/checkpoint_22010.tar'
skipping generation of images for  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210121_133144_3d_l0.5/models/checkpoint_22010.tar


In [35]:
## For multiple steps 
main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/'
fldr='20210227_050213_3dcgan_predict_0.65_m2'

step_list=[27600, 30950, 42620, 43830, 50440, 58320, 58330, 59500, 64670,65600]
for step in step_list:
    for count in range(2): # Repeat inference
        try: 
            op_loc=main_dir+fldr+'/images/'
            ip_fname=main_dir+fldr+'/models/checkpoint_{0}.tar'.format(step)
            f_gen_images(gdict,netG,optimizerG,ip_fname,op_loc,op_strg='inference_%s'%(count),op_size=1000)
        except Exception as e:
            print(e)
            print("skipping ",step)


[Errno 2] No such file or directory: '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210427_103955_bs8_lr0.0001_nodes8/models/checkpoint_10560.tar'
skipping generation of images for  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210427_103955_bs8_lr0.0001_nodes8/models/checkpoint_10560.tar
[Errno 2] No such file or directory: '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210427_103955_bs8_lr0.0001_nodes8/models/checkpoint_10560.tar'
skipping generation of images for  /global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210427_103955_bs8_lr0.0001_nodes8/models/checkpoint_10560.tar
[Errno 2] No such file or directory: '/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/3d/20210427_103955_bs8_lr0.0001_nodes8/models/checkpoint_11600.tar'
skipping generation of images for  /global/cfs/cdirs/

In [ ]:
# fname=op_loc+'inference_spec_epoch-11_step-37040.npy'
# a1=np.load(fname)
# print(a1.shape)